In [3]:
from pathlib import Path
import pandas as pd

import funcoes

In [4]:
parcial_path_absolute = Path.home() / 'OneDrive' / 'Documentos' / 'GitHub'
parcial_path_relative = Path('analise_dados_crimes_seattle')
full_path = parcial_path_absolute / parcial_path_relative

absolute_path_data_raw = full_path / 'data' / 'raw'
absolute_path_data_processed = full_path / 'data' / 'processed'

path_data = absolute_path_data_raw / 'SPD_Crime_Data__2008-Present_20250311.csv'
path_data_info_geo = absolute_path_data_raw / 'SPD_Crime_Data__2008-Present_20250223.csv'

Os dados representam atividades de resposta policial em Seattle.
Cada linha no conjunto de dados é um registro de um Call for Service (CfS), ou seja, uma chamada registrada pelo Centro de Comunicações da Polícia de Seattle (SPD).
As chamadas vêm da comunidade e podem ser desde emergências ativas até pedidos de resolução de problemas.
Os policiais também podem registrar chamadas com base em suas próprias observações no campo.

In [5]:
df = pd.read_csv(path_data, sep=',')
dff = df.copy() #preservando o bruto

In [6]:
dff = funcoes.colunas_lower(dff)

In [7]:
df_info_geografica = pd.read_csv(path_data_info_geo, sep=',')
dff_to_merge_infos = df_info_geografica.copy() #preservando o bruto

In [8]:
dff_to_merge_infos = funcoes.colunas_lower(dff_to_merge_infos)
dff_to_merge_infos = dff_to_merge_infos[['offense id', 'longitude', 'latitude']]

Limpeza de dados

In [9]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1438644 entries, 0 to 1438643
Data columns (total 19 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   report number                   1438644 non-null  object 
 1   report datetime                 1438644 non-null  object 
 2   offense id                      1438644 non-null  int64  
 3   offense date                    1438644 non-null  object 
 4   nibrs group ab                  1438644 non-null  object 
 5   nibrs crime against category    1438644 non-null  object 
 6   offense sub category            1438644 non-null  object 
 7   shooting type group             1438644 non-null  object 
 8   block address                   0 non-null        float64
 9   latitude                        0 non-null        float64
 10  longitude                       0 non-null        float64
 11  beat                            1438644 non-null  object 
 12  

In [10]:
dff.head(3)

,report number,report datetime,offense id,offense date,nibrs group ab,nibrs crime against category,offense sub category,shooting type group,block address,latitude,longitude,beat,precinct,sector,neighborhood,reporting area,offense category,nibrs offense code description,nibrs_offense_code
0,2014-331666,10/04/2014 12:26:00 AM,7685336839,10/04/2014 12:26:00 AM,A,PERSON,ASSAULT OFFENSES,-,NaN,NaN,NaN,N3,North,N,-,NaN,ALL OTHER,Simple Assault,13B
1,2009-095116,03/20/2009 01:58:00 AM,7665902657,03/20/2009 12:50:00 AM,A,PROPERTY,LARCENY-THEFT,-,NaN,NaN,NaN,J2,North,J,-,NaN,PROPERTY CRIME,Theft From Motor Vehicle,23F
2,2014-272495,08/16/2014 09:08:00 PM,7655103418,08/16/2014 02:00:00 PM,A,PROPERTY,MOTOR VEHICLE THEFT,-,NaN,NaN,NaN,K3,West,K,-,NaN,PROPERTY CRIME,Motor Vehicle Theft,240


In [11]:
dff.columns

Index(['report number', 'report datetime', 'offense id', 'offense date',
       'nibrs group ab', 'nibrs crime against category',
       'offense sub category', 'shooting type group', 'block address',
       'latitude', 'longitude', 'beat', 'precinct', 'sector', 'neighborhood',
       'reporting area', 'offense category', 'nibrs offense code description',
       'nibrs_offense_code'],
      dtype='object')

In [12]:
dff = dff.drop(['block address','latitude', 'longitude', 'neighborhood', 'reporting area'], axis=1)

In [13]:
# excluir o que não é considerado crime
dff = dff[dff['nibrs crime against category']!='NOT_A_CRIME']

In [14]:
dff = pd.merge(dff, dff_to_merge_infos, how='left', on='offense id')

In [15]:
dff.head()

,report number,report datetime,offense id,offense date,nibrs group ab,nibrs crime against category,offense sub category,shooting type group,beat,precinct,sector,offense category,nibrs offense code description,nibrs_offense_code,longitude,latitude
0,2014-331666,10/04/2014 12:26:00 AM,7685336839,10/04/2014 12:26:00 AM,A,PERSON,ASSAULT OFFENSES,-,N3,North,N,ALL OTHER,Simple Assault,13B,-122.344675,47.703422
1,2009-095116,03/20/2009 01:58:00 AM,7665902657,03/20/2009 12:50:00 AM,A,PROPERTY,LARCENY-THEFT,-,J2,North,J,PROPERTY CRIME,Theft From Motor Vehicle,23F,-122.385404,47.687759
2,2014-272495,08/16/2014 09:08:00 PM,7655103418,08/16/2014 02:00:00 PM,A,PROPERTY,MOTOR VEHICLE THEFT,-,K3,West,K,PROPERTY CRIME,Motor Vehicle Theft,240,-122.328489,47.602018
3,2009-333330,09/20/2009 11:37:00 PM,7684768411,09/20/2009 01:00:00 AM,A,PROPERTY,"PROPERTY OFFENSES (INCLUDES STOLEN, DESTRUCTION)",-,J2,North,J,ALL OTHER,Destruction/Damage/Vandalism of Property,290,-122.384293,47.678609
4,2011-380459,11/25/2011 11:26:00 AM,7674046661,11/25/2011 12:30:00 AM,A,PROPERTY,LARCENY-THEFT,-,O3,South,O,PROPERTY CRIME,All Other Larceny,23H,-122.319315,47.548336


In [17]:
# 'report datetime'
dff['report datetime'] = funcoes.transformando_colunas_em_datetime(dff, 'report datetime')

# 'offense date
dff['offense date'] = funcoes.transformando_colunas_em_datetime(dff, 'offense date')

c:\Users\tales\OneDrive\Documentos\GitHub\analise_dados_crimes_seattle\src\funcoes\__init__.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(data_frame_target[coluna_target])
c:\Users\tales\OneDrive\Documentos\GitHub\analise_dados_crimes_seattle\src\funcoes\__init__.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(data_frame_target[coluna_target])


In [20]:
path_output_data_cleaned = absolute_path_data_processed / 'SPD_Crime_data_cleaned.csv'
dff.to_csv(path_output_data_cleaned, index=False)

In [21]:
import numpy as np

n_partes = 3
dfs_divididos = np.array_split(dff, n_partes)

for i, parte in enumerate(dfs_divididos):
    parte.to_csv((absolute_path_data_processed / f'split_SPD_Crime_data_cleaned_{i+1}.csv'), index=False)


c:\Users\tales\OneDrive\Documentos\GitHub\analise_dados_crimes_seattle\venv_crimes_seatle\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
